In [ ]:
!curl https://colab.chainer.org/install | sh -

In [ ]:
! nvcc --version

In [ ]:
! pip install pycuda

In [ ]:
# check to make sure PyCUDA installed properly
import pycuda.autoinit

In [ ]:
!pip install cython

In [ ]:
!# install scikits-cuda from github as the pip version cant find cusolver
!pip install scikit-cuda

In [ ]:
! pip install progressbar

In [ ]:
#!git clone https://github.com/CameronDallas5000/openpiv-python.git
!git clone https://github.com/OpenPIV/openpiv-python-gpu.git
import os, sys
os.chdir('openpiv-python-gpu')
sys.path.append(os.getcwd())
!python setup.py build_ext --inplace
!python setup.py install

In [ ]:
import openpiv

In [ ]:
from openpiv import tools, process, validation, filters, scaling, pyprocess
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [ ]:
!wget https://raw.githubusercontent.com/CameronDallas5000/openpiv-python/master/openpiv/examples/tutorial-part1/exp1_001_a.bmp
!wget https://raw.githubusercontent.com/CameronDallas5000/openpiv-python/master/openpiv/examples/tutorial-part1/exp1_001_b.bmp
!ls

In [ ]:
# from google.colab import files
# files.upload() # choose exp1_001_b.bmp and _c.bmp

In [ ]:
import imageio as io
frame_a  = io.imread( 'exp1_001_a.bmp' )
frame_b  = io.imread( 'exp1_001_b.bmp' )

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,8))
ax[0].imshow(frame_a,cmap=plt.cm.gray)
ax[1].imshow(frame_b,cmap=plt.cm.gray)

# OpenPIV Python version

In [ ]:
winsize = 32 # pixels
searchsize = 64  # pixels, search in image B
overlap = 12 # pixels
dt = 0.02 # sec

In [ ]:
u, v, sig2noise = pyprocess.extended_search_area_piv( frame_a.astype(np.int32), frame_b.astype(np.int32), 
                                                     window_size=winsize, overlap=overlap, dt=dt, 
                                                     search_area_size=searchsize, sig2noise_method='peak2peak' )

In [ ]:
x, y = pyprocess.get_coordinates( image_size=frame_a.shape, window_size=searchsize, overlap=overlap )

In [ ]:
tools.save(x, y, u, v, np.zeros_like(u), 'exp1_001.txt' ) # no masking, all values are valid

# OpenPIV Python-GPU version

In [ ]:
from progressbar import *


In [ ]:
import openpiv.gpu_process
# reload(openpiv.gpu_process)

In [ ]:
# gpu code parametes
min_window_size = 32
overlap_ratio = 0.5
coarse_factor = 1
nb_iter_max = 2

In [ ]:
x, y, u, v, mask = openpiv.gpu_process.WiDIM( frame_a.astype(np.int32), frame_b.astype(np.int32), np.ones(frame_a.shape, dtype=np.int32),
                                                     min_window_size, 
                                                     overlap_ratio,
                                                     coarse_factor,
                                                     dt,
                                                     nb_iter_max = nb_iter_max)

In [ ]:
tools.save(x, y, u, v, np.zeros_like(u), 'exp1_001_gpu.txt' )

# OpenPIV Python-CPU (Cython) version

In [ ]:
x, y, u, v, mask = openpiv.process.WiDIM( frame_a.astype(np.int32), frame_b.astype(np.int32), np.ones(frame_a.shape, dtype=np.int32),
                                                     min_window_size, 
                                                     overlap_ratio,
                                                     coarse_factor,
                                                     dt,
                                                     nb_iter_max = nb_iter_max)

In [ ]:
tools.save(x, y, u, v, np.zeros_like(u), 'exp1_001_cpu.txt' )

In [ ]:
tools.display_vector_field('exp1_001.txt', scale=5000, width=0.0025)
tools.display_vector_field('exp1_001_cpu.txt', scale=5000, width=0.0025)
tools.display_vector_field('exp1_001_gpu.txt', scale=5000, width=0.0025)